In [ ]:
import os
import pandas as pd
import geopandas as gpd
from access import Access, weights

In [ ]:
# Census tracts shapefile from https://www.nhgis.org
census_tracts = gpd.read_file('/data/nhgis0001_shapefile_tl2019_us_tract_2019/US_tract_2019.shp')
census_tracts = census_tracts.to_crs(epsg=4326)
census_tracts = census_tracts.rename(columns={'GEOID': 'geoid'})

In [ ]:
# travel cost matrix
cost_matrix = pd.read_csv('/data/combined.csv') 
cost_matrix = cost_matrix.loc[cost_matrix['origin'] != 'origin'] 

In [ ]:
professions = ['psychiatrists', 'therapists']

for profession in professions:
    
    # load provider coordinates csv
    if profession == "psychiatrists": providers = pd.read_csv('/data/psychiatrists.csv')
    if profession == "therapists": providers = pd.read_csv('/data/therapists.csv')
        
    if profession in ['psychiatrists']:        
        population = pd.read_csv('/data/population.csv')
    else:
        population = pd.read_csv('/data/population_therapists.csv')
     
    population = population.rename(columns={'origin': 'geoid'})
        
    providers_geom = gpd.GeoDataFrame(providers, crs='epsg:4326',
                                    geometry = gpd.points_from_xy(providers.lon, providers.lat))
    providers_geom = providers_geom.to_crs(epsg=4326)
    
    providers_geom = gpd.sjoin(providers_geom, census_tracts[['geoid', 'geometry']], how='inner', op='intersects')
    providers_geom = providers_geom.drop(columns=['index_right'])
    
    for col in providers_geom.columns[4:5]: 
        providers_geom[col] = pd.to_numeric(providers_geom[col], errors='coerce')

    aggregated_values_per_tract = pd.DataFrame()
    aggregated_values_per_tract['value'] = providers_geom.groupby(['geoid'])[['value']].agg('sum')
    aggregated_values_per_tract['geoid'] = aggregated_values_per_tract.index
    
    provider_access = Access(demand_df = population, 
                            demand_index = 'geoid', 
                            demand_value = 'total_pop',
                            supply_df    = aggregated_values_per_tract, 
                            supply_index = 'geoid',
                            supply_value = 'value',
                            cost_df      = cost_matrix, 
                            cost_origin  = 'origin',
                            cost_dest    = 'destination',
                            cost_name    = 'minutes')
    
    fn_gaussian = weights.gaussian(sigma = 30)
    provider_access.three_stage_fca(name = '3sfca', weight_fn = fn_gaussian)
    temp_df = provider_access.access_df.drop(columns=['total_pop']).rename(columns={'3sfca_value': 'combined_3sfca'})

    # saving csv
    temp_df.to_csv('/data/'+profession+'_3sfca.csv', index=True)